# Introduction - Customer Churn Prediction notebook
In this notebook, we illustrate how you can train a model for Churn Prediction using scikit learn. After training the model, you step through the instructions to deploy the model using Watson Machine Learning.

This notebook is a variation of the original notebook reference in this github repo: https://github.com/elenalowery/cpd4_demo/blob/master/assets/jupyterlab/Predict_Customer_Churn_CPD4.ipynb


In [1]:
# Install required Python modules
!pip install sklearn-pandas


## Step 1: Review Use Case
The analytics use case implemented in this notebook is telco churn prediction. It is a simple use case which illustrates typical process for model development and deployment using Cloud Pak for Data.

In [9]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

db2cld_metadata = wslib.get_connection("db2cld")

import os, ibm_db, ibm_db_dbi as dbi, pandas as pd

db2cld_dsn = 'DATABASE={};HOSTNAME={};PORT={};PROTOCOL=TCPIP;UID={uid};PWD={pwd};SECURITY=SSL'.format(
    db2cld_metadata['database'],
    db2cld_metadata['host'],
    db2cld_metadata.get('port', 50000),
    uid=db2cld_metadata['username'],
    pwd=db2cld_metadata['password']
)

db2cld_connection = dbi.connect(db2cld_dsn)

# NOTE:
#  A row limit has been applied to the query to enable sample previewing.
#  Adjust the display message and query as needed by editing the following lines:
from IPython.core.display import display, HTML
display(HTML("A row limit of 5000 has been applied to the query to enable sample previewing. If the data set is larger, only the first 5000 rows will be loaded."))
query = 'SELECT * FROM "XCV64422"."INITIAL_TRAINING_DATA" FETCH FIRST 5000 ROWS ONLY'

data_df_6 = pd.read_sql_query(query, con=db2cld_connection)
data_df_6.head()

# After use, close the database connection with the following code:
# db2cld_connection.close()


,COLUMN_0,ID,LONGDISTANCE,INTERNATIONAL,LOCAL,DROPPED,PAYMETHOD,LOCALBILLTYPE,LONGDISTANCEBILLTYPE,USAGE,RATEPLAN,GENDER,STATUS,CHILDREN,ESTINCOME,CAROWNER,AGE,CHURN
0,433,2046,8,0,13,0,CH,Budget,Standard,21,2,F,M,2,56187.0,N,46.000000,F
1,1342,851,23,0,51,0,Auto,FreeLocal,Standard,74,3,F,S,0,92564.1,Y,49.133333,F
2,1073,3633,16,0,10,0,CC,FreeLocal,Intnl_discount,26,3,F,M,2,55860.0,Y,14.573333,F
3,1249,596,16,0,56,0,CC,Budget,Intnl_discount,72,4,F,M,0,32703.1,Y,64.920000,F
4,1292,740,4,0,6,0,CC,Budget,Intnl_discount,11,2,F,M,0,14932.5,N,46.000000,F


In [10]:
# COPY the dataFrame into a new dataFrame called *data*
# Note that when you insert the data, it may get written to a different dataFrame name; in this example, it is assumed the dataFrame is data_df_1
data=data_df_6.copy()

In [11]:
# List all the columns
print(data.columns)

Index(['COLUMN_0', 'ID', 'LONGDISTANCE', 'INTERNATIONAL', 'LOCAL', 'DROPPED',
       'PAYMETHOD', 'LOCALBILLTYPE', 'LONGDISTANCEBILLTYPE', 'USAGE',
       'RATEPLAN', 'GENDER', 'STATUS', 'CHILDREN', 'ESTINCOME', 'CAROWNER',
       'AGE', 'CHURN'],
      dtype='object')


In [12]:
# Keep only the columns that are relevant for churn prediction
data = data[['ID', 'LONGDISTANCE', 'INTERNATIONAL', 'LOCAL', 'DROPPED', 'PAYMETHOD', 'LOCALBILLTYPE', 'LONGDISTANCEBILLTYPE', 'USAGE', 'RATEPLAN', 'GENDER','STATUS', 'CHILDREN', 'ESTINCOME', 'CAROWNER', 'AGE', 'CHURN']]
data.head()


,ID,LONGDISTANCE,INTERNATIONAL,LOCAL,DROPPED,PAYMETHOD,LOCALBILLTYPE,LONGDISTANCEBILLTYPE,USAGE,RATEPLAN,GENDER,STATUS,CHILDREN,ESTINCOME,CAROWNER,AGE,CHURN
0,2046,8,0,13,0,CH,Budget,Standard,21,2,F,M,2,56187.0,N,46.000000,F
1,851,23,0,51,0,Auto,FreeLocal,Standard,74,3,F,S,0,92564.1,Y,49.133333,F
2,3633,16,0,10,0,CC,FreeLocal,Intnl_discount,26,3,F,M,2,55860.0,Y,14.573333,F
3,596,16,0,56,0,CC,Budget,Intnl_discount,72,4,F,M,0,32703.1,Y,64.920000,F
4,740,4,0,6,0,CC,Budget,Intnl_discount,11,2,F,M,0,14932.5,N,46.000000,F


## Step 2: Build the Random Forest model

In [13]:
import pandas as pd
import sklearn
pd.options.display.max_columns = 999

import warnings
warnings.filterwarnings('ignore')

from scipy.stats import chi2_contingency,ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, roc_curve, roc_auc_score

import numpy as np

import urllib3, requests, json

In [14]:
#convert CHURN to 1/0
le = LabelEncoder()
data.loc[:,'CHURN']= le.fit_transform(data.loc[:,'CHURN'])
data.head()

,ID,LONGDISTANCE,INTERNATIONAL,LOCAL,DROPPED,PAYMETHOD,LOCALBILLTYPE,LONGDISTANCEBILLTYPE,USAGE,RATEPLAN,GENDER,STATUS,CHILDREN,ESTINCOME,CAROWNER,AGE,CHURN
0,2046,8,0,13,0,CH,Budget,Standard,21,2,F,M,2,56187.0,N,46.000000,0
1,851,23,0,51,0,Auto,FreeLocal,Standard,74,3,F,S,0,92564.1,Y,49.133333,0
2,3633,16,0,10,0,CC,FreeLocal,Intnl_discount,26,3,F,M,2,55860.0,Y,14.573333,0
3,596,16,0,56,0,CC,Budget,Intnl_discount,72,4,F,M,0,32703.1,Y,64.920000,0
4,740,4,0,6,0,CC,Budget,Intnl_discount,11,2,F,M,0,14932.5,N,46.000000,0


In [15]:
# define the label and features
y = np.float32(data.CHURN)
x = data.drop(['CHURN'], axis = 1)

In [16]:
x.columns

Index(['ID', 'LONGDISTANCE', 'INTERNATIONAL', 'LOCAL', 'DROPPED', 'PAYMETHOD',
       'LOCALBILLTYPE', 'LONGDISTANCEBILLTYPE', 'USAGE', 'RATEPLAN', 'GENDER',
       'STATUS', 'CHILDREN', 'ESTINCOME', 'CAROWNER', 'AGE'],
      dtype='object')

In [17]:
# Apply the LabelEncoder to encode the input features in numeric form where applicable
from sklearn_pandas import DataFrameMapper

mapper = DataFrameMapper(
    [('GENDER', LabelEncoder()),
     ('STATUS', LabelEncoder()),
     ('CHILDREN', None),
     ('ESTINCOME',None),
     ('CAROWNER', LabelEncoder()),
     ('AGE',None),
     ('LONGDISTANCE',None),
     ('INTERNATIONAL',None),
     ('LOCAL',None),
     ('DROPPED',None),
     ('PAYMETHOD',LabelEncoder()),
     ('LOCALBILLTYPE',LabelEncoder()),
     ('LONGDISTANCEBILLTYPE',LabelEncoder()),
     ('USAGE',None),
     ('RATEPLAN',None)
    ]
)

In [18]:
# split the data to training and testing set
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [19]:
# fit the model

import sklearn.pipeline
from sklearn.preprocessing import OneHotEncoder

random_forest = RandomForestClassifier()
steps = [('mapper', mapper),('RandonForestClassifier', random_forest)]
pipeline = sklearn.pipeline.Pipeline(steps)
model=pipeline.fit( X_train, y_train )
model

Pipeline(steps=[('mapper',
                 DataFrameMapper(drop_cols=[],
                                 features=[('GENDER', LabelEncoder()),
                                           ('STATUS', LabelEncoder()),
                                           ('CHILDREN', None),
                                           ('ESTINCOME', None),
                                           ('CAROWNER', LabelEncoder()),
                                           ('AGE', None),
                                           ('LONGDISTANCE', None),
                                           ('INTERNATIONAL', None),
                                           ('LOCAL', None), ('DROPPED', None),
                                           ('PAYMETHOD', LabelEncoder()),
                                           ('LOCALBILLTYPE', LabelEncoder()),
                                           ('LONGDISTANCEBILLTYPE',
                                            LabelEncoder()),
                               

In [20]:
### call pipeline.predict() on your X_test data to make a set of test predictions
y_prediction = pipeline.predict( X_test )

### test your predictions using sklearn.classification_report()

report = sklearn.metrics.classification_report( y_test, y_prediction )
### and print the report
print(report)

              precision    recall  f1-score   support

         0.0       0.99      0.96      0.98       111
         1.0       0.95      0.99      0.97        79

    accuracy                           0.97       190
   macro avg       0.97      0.98      0.97       190
weighted avg       0.97      0.97      0.97       190

